Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Радует, что ноутбук хорошо структурирован. Приятно проверять такие работы.
* Советую использовать маркеры, жирный шрифт итд в текстовых комментариях. Хорошо структурированную информацию намного легче воспринимать.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* В работе я оставил несколько советов. Буду рад, если ты учтешь их.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

### <font color='orange'>Общее впечатление (ревью 2)</font>
* Для удобства все новые комментарии обозначены фразой "ревью 2".
* Удачи в доработке!

### <font color='orange'>Общее впечатление (ревью 3)</font>
* После исправлений проект улучшился и теперь он может быть зачтен.
* Новые комментарии обозначены фразой "ревью 3".
* Удачи в дальнейшем обучении и следующих работах!

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostRegressor, Pool
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import time


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [64]:
df = pd.read_csv('/datasets/autos.csv')
df.head(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


Мы видим, что в данных есть пропуски, но библиотека CatBoost, которую мы планируем использовать умеет с ними работать, но библиотека LightGBM, которую нам также нужно использовать - нет. Поэтому заменим пропуски на моды, те на самые часто встречающиеся значения признака.

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены корректно.
</div>

In [66]:
#df['VehicleType'] = df['VehicleType'].fillna(df['VehicleType'].mode().values[0])
#df['Gearbox'] = df['Gearbox'].fillna(df['Gearbox'].mode().values[0])
#df['Model'] = df['Model'].fillna(df['Model'].mode().values[0])
#df['FuelType'] = df['FuelType'].fillna(df['FuelType'].mode().values[0])
#df['NotRepaired'] = df["NotRepaired"].fillna('no')

In [67]:
df['VehicleType'] = df['VehicleType'].fillna('UNKNOWN')
df['Gearbox'] = df['Gearbox'].fillna('UNKNOWN')
df['Model'] = df['Model'].fillna('UNKNOWN')
df['FuelType'] = df['FuelType'].fillna('UNKNOWN')
df['NotRepaired'] = df["NotRepaired"].fillna('UNKNOWN')

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Заменила на UNKNOWN
</div>

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          354369 non-null object
RegistrationYear     354369 non-null int64
Gearbox              354369 non-null object
Power                354369 non-null int64
Model                354369 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             354369 non-null object
Brand                354369 non-null object
NotRepaired          354369 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


<div class="alert alert-block alert-success">
<b>Успех:</b> В целом, это допустимое решение.
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Однако, я бы заполнил уникальной строкой "UNKNOWN", так было бы безопаснее.
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> ОК.
</div>

Так же мы видим, что есть переменные, которые с малой вероятностью влияют на интересуюий признак цены машины и скорее важны для самого приложения. Например, число фотографий или дата последней активности пользователя. Я думаю, их лучше сразу удалить, чтобы не усложнять работу наших будущих моделей.

In [69]:
df = df.drop(['DateCrawled','DateCreated','LastSeen','NumberOfPictures','PostalCode'],axis = 1)

<div class="alert alert-block alert-success">
<b>Успех:</b> Согласен, что даты, константный признак и индекс нам не нужны.
</div>

In [70]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645
std,4514.158514,90.227958,189.850405,37905.341530,3.726421
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


Видно, что для переменной цены и мощности стандартные отклонения сопоставимы со средними значениями, это свидетельство сильного разброса данных. Возможно, в нашей выборке присуствуют выбросы, но может и правда для данных переменных характерен большой разброс. Стоило бы уточнить этот вопрос у коллег. Но помнить, что если модели будут показывать плохой результат, можно попробовать их обучить без выбросов.

### Преобразование признаков

В нашей выборке много категориальных признаков, таких как марка автомобиля или тип топлива, поэтому перед обучением моделей стоит их преобразовать. 

In [71]:
o_features = df.drop('Price', axis=1)

encoder = OrdinalEncoder()
cat_columns = ['VehicleType','FuelType','Brand','Model','NotRepaired']
cat_features = o_features[cat_columns]
cat_features = pd.DataFrame(encoder.fit_transform(cat_features),
                                columns=cat_features.columns, 
                                index=cat_features.index)
o_features = o_features.copy()
for column in cat_columns:
    o_features[column] = cat_features[column]

ohe_features = pd.get_dummies(o_features, drop_first=True)

target = df['Price']

In [72]:
o_features.head(3)

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
0,0.0,1993,manual,0,117.0,150000,0,7.0,38.0,0.0
1,3.0,2011,manual,190,26.0,125000,5,3.0,1.0,2.0
2,7.0,2004,auto,163,118.0,125000,8,3.0,14.0,0.0


In [73]:
cat_features.head(3)

,VehicleType,FuelType,Brand,Model,NotRepaired
0,0.0,7.0,38.0,117.0,0.0
1,3.0,3.0,1.0,26.0,2.0
2,7.0,3.0,14.0,118.0,0.0


In [74]:
ohe_features.head(3)

,VehicleType,RegistrationYear,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,Gearbox_auto,Gearbox_manual
0,0.0,1993,0,117.0,150000,0,7.0,38.0,0.0,0,1
1,3.0,2011,190,26.0,125000,5,3.0,1.0,2.0,0,1
2,7.0,2004,163,118.0,125000,8,3.0,14.0,0.0,1,0


<div class="alert alert-block alert-success">
<b>Успех:</b> Данные закодированы хорошо.
</div>

Разделим наши данные на тренировочную и тестовую выборки.

In [75]:
o_features_train, o_features_test, target_train, target_test = train_test_split(
    o_features, target, test_size=0.25, random_state=12345)

o_features_train = o_features.loc[o_features_train.index, :]
o_features_test = o_features.loc[o_features_test.index, :]

ohe_features_train = ohe_features.loc[o_features_train.index, :]
ohe_features_test = ohe_features.loc[o_features_test.index, :]


<div class="alert alert-block alert-success">
<b>Успех:</b> Разбиение было сделано верно, молодец!
</div>

Так как мы видели, что у данных большой разброс по целевому признаку проверим не сильно ли отличаются между собй значения попавшие в тестовую и тренировочную выборки.

In [76]:
print(target_test.mean())
print(target_train.mean())

4426.531035183367
4413.365318915176


## Обучение моделей

### Linear Regression

In [77]:
%%time

regressor = LinearRegression()
cv_RMSE_LR = (cross_val_score(regressor, 
                             ohe_features_train, 
                             target_train, 
                             cv=5, 
                             scoring='neg_mean_squared_error').mean() * -1) ** 0.5
cv_RMSE_LR

CPU times: user 735 ms, sys: 514 ms, total: 1.25 s
Wall time: 1.23 s


3925.7895060567407

### Decision Tree

In [78]:
%%time

regressor = DecisionTreeRegressor() 
max_depth_list = [x for x in range(2, 20)]
hyperparams = [{'criterion':['mse'], 
                'max_depth':max_depth_list, 
                'random_state':[12345]}]

clf = GridSearchCV(regressor, hyperparams, scoring='neg_mean_squared_error')
clf.fit(ohe_features_train, target_train)
print(clf.best_params_)

means = clf.cv_results_['mean_test_score']
cv_RMSE_DTR_ohe = (max(means)*-1) ** 0.5
cv_RMSE_DTR_ohe

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'criterion': 'mse', 'max_depth': 12, 'random_state': 12345}
CPU times: user 21.6 s, sys: 0 ns, total: 21.6 s
Wall time: 21.9 s


2118.201180601501

<div class="alert alert-block alert-success">
<b>Успех:</b> Радует, что для подбора параметров использовался метод основанный на кросс-валидации с корректной метрикой.
</div>

### Catboost

In [79]:
cat_columns = ['VehicleType','FuelType','Brand','Model','NotRepaired']
ohe_features_train_cb = o_features_train[cat_columns].astype(int)
ohe_features_test_cb = o_features_test[cat_columns].astype(int)
train_pool = Pool(ohe_features_train_cb, target_train, cat_features = cat_columns)
test_pool = Pool(ohe_features_test_cb, target_test, cat_features = cat_columns)

In [80]:
%%time

model_cb = CatBoostRegressor(iterations=200,learning_rate=0.1,loss_function='RMSE') 

model_cb.fit(train_pool)

0:	learn: 4367.7936159	total: 362ms	remaining: 1m 12s
1:	learn: 4248.3151741	total: 668ms	remaining: 1m 6s
2:	learn: 4148.4176767	total: 1.06s	remaining: 1m 9s
3:	learn: 4063.1129488	total: 1.27s	remaining: 1m 2s
4:	learn: 3991.7693299	total: 1.56s	remaining: 1m
5:	learn: 3932.9736597	total: 1.86s	remaining: 1m
6:	learn: 3882.7836252	total: 2.06s	remaining: 56.9s
7:	learn: 3841.5772118	total: 2.36s	remaining: 56.6s
8:	learn: 3805.7796845	total: 2.57s	remaining: 54.5s
9:	learn: 3773.6861133	total: 2.86s	remaining: 54.3s
10:	learn: 3747.1206817	total: 3.06s	remaining: 52.6s
11:	learn: 3725.1778033	total: 3.36s	remaining: 52.6s
12:	learn: 3708.2297936	total: 3.65s	remaining: 52.6s
13:	learn: 3690.4614047	total: 3.86s	remaining: 51.3s
14:	learn: 3674.7146445	total: 4.16s	remaining: 51.3s
15:	learn: 3661.3324562	total: 4.46s	remaining: 51.3s
16:	learn: 3649.8854705	total: 4.76s	remaining: 51.3s
17:	learn: 3640.7693162	total: 5.06s	remaining: 51.1s
18:	learn: 3632.5123106	total: 5.36s	remain

In [81]:
# %%time

#regressor = CatBoostRegressor() 
#hyperparams = [{'learning_rate':[0.1, 0.5, 0.8],
#                'random_state':[12345],
#                'verbose':[False]}]

#clf = GridSearchCV(regressor, hyperparams, scoring='neg_mean_squared_error')
#clf.fit(ohe_features_train, target_train)

#print(clf.best_params_)

#means = clf.cv_results_['mean_test_score']
#cv_RMSE_cb_ohe = (max(means)*-1) ** 0.5
#cv_RMSE_cb_ohe

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Я не понимаю, что нужно тут использовать. Я пыталась использовать Pool, он все время выдает ошибки, тем более у меня нет информации о весах, а писать ее от рандома странно. В документации Catboost нет примеров. Короче помоги, пожалуйста, я вообще не понимаю, что тут нужно делать и зачем. 
</div>

<div class="alert alert-block alert-info">
<b>Совет (ревью 2): </b> <a href="https://catboost.ai/docs/concepts/python-usages-examples.html#binary-classification">Тут</a> есть пример.
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 3):</b> Верно!
</div>

### LGBM Regressor

In [82]:
ohe_features_train_lgbm = ohe_features_train.astype('category')
ohe_features_test_lgbm = ohe_features_test.astype('category')
ohe_features_train_lgbm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265776 entries, 25945 to 217570
Data columns (total 11 columns):
VehicleType          265776 non-null category
RegistrationYear     265776 non-null category
Power                265776 non-null category
Model                265776 non-null category
Kilometer            265776 non-null category
RegistrationMonth    265776 non-null category
FuelType             265776 non-null category
Brand                265776 non-null category
NotRepaired          265776 non-null category
Gearbox_auto         265776 non-null category
Gearbox_manual       265776 non-null category
dtypes: category(11)
memory usage: 5.6 MB


In [83]:
%%time

regressor = LGBMRegressor() 
hyperparams = [{'num_leaves':[31, 100, 200], 
                'learning_rate':[0.1, 0.5, 0.8],
                'random_state':[12345]}]

clf = GridSearchCV(regressor, hyperparams, scoring='neg_mean_squared_error')
clf.fit(ohe_features_train_lgbm, target_train)
print(clf.best_params_)


means = clf.cv_results_['mean_test_score']
cv_RMSE_lgbm_ohe = (max(means)*-1) ** 0.5
cv_RMSE_lgbm_ohe

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'learning_rate': 0.1, 'num_leaves': 200, 'random_state': 12345}
CPU times: user 3min 57s, sys: 1 s, total: 3min 58s
Wall time: 4min 2s


1717.6301942016187

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> У моделей Catboost и lightGBM существуют внутренние механизмы обработки категориальных признаков. В данном случае нужно использовать их. Более подробную информацию можно найти в интернете.
    <br> Подскажу с lightGBM: достаточно изменить тип колонок с категориальными признаками на соответствующий: .astype('category').
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Заменила на для lightGBM тип признаков на category.
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка (ревью 2):</b> Ты не используешь ohe_features_train_lgbm .
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 3):</b> Теперь все ОК.
</div>

## Анализ моделей

### Linear Regression

In [84]:
%%time
start_time = time.time()
model_lr = LinearRegression()
model_lr.fit(ohe_features_train, target_train)

time_lr = (time.time() - start_time)
time_lr

CPU times: user 99.8 ms, sys: 42.4 ms, total: 142 ms
Wall time: 119 ms


0.11901402473449707

In [85]:
%%time
start_time = time.time()
target_predict_lr = model_lr.predict(ohe_features_test)
RMSE_lr = mean_squared_error(target_test, target_predict_lr) ** 0.5
RMSE_lr

time_lr_pred = (time.time() - start_time)
time_lr_pred

CPU times: user 28.6 ms, sys: 4.92 ms, total: 33.5 ms
Wall time: 6.99 ms


0.0068895816802978516

### Decision Tree

In [86]:
%%time
start_time = time.time()
model_dt = DecisionTreeRegressor(criterion='mse', 
                              max_depth=15, 
                              random_state=12345) 
model_dt.fit(ohe_features_train, target_train)

time_dt = (time.time() - start_time)
time_dt

CPU times: user 796 ms, sys: 44.7 ms, total: 840 ms
Wall time: 865 ms


0.8650081157684326

In [87]:
%%time
start_time = time.time()
target_predict_dt = model_dt.predict(ohe_features_test)
RMSE_dt = mean_squared_error(target_test, target_predict_dt) ** 0.5
RMSE_dt

time_dt_pred = (time.time() - start_time)
time_dt_pred

CPU times: user 16.5 ms, sys: 4.12 ms, total: 20.6 ms
Wall time: 27.1 ms


0.027035951614379883

### CatBoost

In [88]:
#%%time
#start_time = time.time()
#model_cb = CatBoostRegressor(learning_rate=0.5, 
#                          random_state=12345, 
#                          verbose=False) 
#model_cb.fit(ohe_features_train, target_train)

#time_cb = (time.time() - start_time)
#time_cb

In [94]:
%%time
start_time = time.time()
model_cb = CatBoostRegressor(iterations=200,learning_rate=0.1,loss_function='RMSE') 

model_cb.fit(train_pool)
time_cb= (time.time() - start_time)
time_cb

0:	learn: 4367.7936159	total: 404ms	remaining: 1m 20s
1:	learn: 4248.3151741	total: 709ms	remaining: 1m 10s
2:	learn: 4148.4176767	total: 1.1s	remaining: 1m 12s
3:	learn: 4063.1129488	total: 1.31s	remaining: 1m 4s
4:	learn: 3991.7693299	total: 1.6s	remaining: 1m 2s
5:	learn: 3932.9736597	total: 1.9s	remaining: 1m 1s
6:	learn: 3882.7836252	total: 2.1s	remaining: 58s
7:	learn: 3841.5772118	total: 2.4s	remaining: 57.6s
8:	learn: 3805.7796845	total: 2.7s	remaining: 57.2s
9:	learn: 3773.6861133	total: 2.9s	remaining: 55.1s
10:	learn: 3747.1206817	total: 3.19s	remaining: 54.9s
11:	learn: 3725.1778033	total: 3.4s	remaining: 53.3s
12:	learn: 3708.2297936	total: 3.7s	remaining: 53.2s
13:	learn: 3690.4614047	total: 4s	remaining: 53.1s
14:	learn: 3674.7146445	total: 4.29s	remaining: 53s
15:	learn: 3661.3324562	total: 4.69s	remaining: 54s
16:	learn: 3649.8854705	total: 5.09s	remaining: 54.8s
17:	learn: 3640.7693162	total: 5.39s	remaining: 54.5s
18:	learn: 3632.5123106	total: 5.69s	remaining: 54.2s

55.437947511672974

In [90]:
%%time
start_time = time.time()
target_predict_cb = model_cb.predict(test_pool)
RMSE_cb = mean_squared_error(target_test, target_predict_cb) ** 0.5
RMSE_cb
time_cb_pred = (time.time() - start_time)
time_cb_pred

CPU times: user 95 ms, sys: 8.34 ms, total: 103 ms
Wall time: 98 ms


0.09791922569274902

### LGBMRegressor

In [91]:
%%time
start_time = time.time()
model_lgbm = LGBMRegressor(learning_rate=0.1, 
                      num_leaves=200, 
                      random_state=12345)
model_lgbm.fit(ohe_features_train_lgbm, target_train)

time_lgbm = (time.time() - start_time)
time_lgbm

CPU times: user 13.8 s, sys: 62 ms, total: 13.9 s
Wall time: 14.1 s


14.096331119537354

In [92]:
%%time
start_time = time.time()
target_predict_lgbm = model_lgbm.predict(ohe_features_test_lgbm)
RMSE_lgbm = mean_squared_error(target_test, target_predict_lgbm) ** 0.5
RMSE_lgbm
time_lgbm_pred = (time.time() - start_time)
time_lgbm_pred

CPU times: user 1.53 s, sys: 4.16 ms, total: 1.54 s
Wall time: 1.59 s


1.5921664237976074

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что отдельно замеряла время обучения и время предсказания. Тестирование было сделано верно.
</div>

In [95]:
index = ['LinearRegression',
         'DecisionTreeRegressor',
         'CatBoostRegressor',
         'LGBMRegressor']

data = {'RMSE': [RMSE_lr, RMSE_dt, RMSE_cb, RMSE_lgbm], 'Learning time': [time_lr, time_dt, time_cb, time_lgbm], 'Prediction time': [time_lr_pred, time_dt_pred, time_cb_pred, time_lgbm_pred]}


result = pd.DataFrame(data=data, index=index)
result

,RMSE,Learning time,Prediction time
LinearRegression,3921.362979,0.119014,0.006890
DecisionTreeRegressor,2067.855966,0.865008,0.027036
CatBoostRegressor,3452.081196,55.437948,0.097919
LGBMRegressor,1702.225594,14.096331,1.592166


<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно было добавить время предсказания в эту тубалицу.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Добавила столбец времени предсказания.
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Отлично!
</div>

### Вывод

Мы подобрали оптимальные параметры и обучили 4 модели. Если мы хотим достигнуть компромиса между скоростью и точностью, то нам стоит выбрать модель Decision Tree, так как она значительно быстрее обучается, чем CatBoostRegressor и LGBMRegressor, но не сильно отличаются по точности. Если же мы сменим приоритеты в сторону точности, то чтоит выбрать модель LGBMRegressor. 
Если бы заказчика не устроила точность нашей модели, то в потенциале можно было бы попробовать подобрать другие гиперпараметры, просто на это потребовалось дополнительное время. Так же для улучшения качества модели можно было бы попробовать исключить значения переменной цена, которые сильно больше стандартного отклонения, так же можно было бы поробовать другую систуму кодирования категориальных признаков. 

<div class="alert alert-block alert-success">
<b>Успех:</b> Анализ с равнение проведены.
</div>

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей